In [1]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import re
import time
import json
import datetime
import unicodedata

In [2]:
url_general = 'https://maldita.es/malditobulo/1'
html_general = requests.get(url_general).text
soup_general = BeautifulSoup(html_general, 'html.parser')

In [3]:
clase_nav_bar = 'bg-white text-gray-800 px-3 py-2 text-lg border-gray-400 font-bold'

pags_link = soup_general.find('span', attrs={'class' : clase_nav_bar})('a')[0].get('href', None)
num_pags = int(pags_link.split('/')[-1])
print(num_pags)

566


In [4]:
def replace_characters(text):
    
    text_aux = ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^\w\s]', '', text_aux.lower().replace(u'\xa0', u' ').replace(u'\n', u' ')).split(" ")

In [5]:
def replace_characters_title(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')

In [6]:
bulos = {}

clase_new = 'section-card-headline'
cont = 0
for i in range(1, num_pags+1):
    url = 'https://maldita.es/malditobulo/'+str(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    for div in soup.find_all('div',attrs={'class' : clase_new}):
        try:
            a = div('a')[0]
            title = a.text
            link = a.get('href', None)
            words = dict()
            
            # Obtener la fecha de la noticia
            if "https://maldita.es/malditobulo/" in link:
                date = link.replace("https://maldita.es/malditobulo/", "")[0:8]
            elif "https://migracion.maldita.es/" in link:
                date = link.replace("https://migracion.maldita.es/", "")[0:8]
            else:
                date = link.replace("https://alimentacion.maldita.es/mitos-bulos/", "")[0:8]
            
            formated_date =  datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y/%m/%d')
            
            html_new = requests.get(link, allow_redirects=False).text

            soup_new = BeautifulSoup(html_new, 'html.parser')

            title_list = replace_characters(title.lower())


            for w in title_list:
                if w != '':
                    if w in words:
                        words[w] += 1
                    else:
                        words[w] = 1

                    
            new_text = soup_new.find('div',attrs={'id' : "article-text"})            
            
            text_list = new_text('p') + new_text('li')

            
            for text in text_list:
                if(not text.get_text().startswith("Una publicación compartida de")):
                    for w in replace_characters(text.get_text()):
                        if w != '':
                            if w in words:
                                words[w] += 1
                            else:
                                words[w] = 1


            bulos[cont] = {"titulo" : replace_characters_title(title), "link" : link, "date" : formated_date, "words_count" : words}
        except:
            print("No se ha podido obtener la informacion de la noticia", i)
        cont += 1

No se ha podido obtener la informacion de la noticia 75
No se ha podido obtener la informacion de la noticia 176
No se ha podido obtener la informacion de la noticia 287
No se ha podido obtener la informacion de la noticia 332
No se ha podido obtener la informacion de la noticia 439
No se ha podido obtener la informacion de la noticia 455
No se ha podido obtener la informacion de la noticia 501
No se ha podido obtener la informacion de la noticia 501
No se ha podido obtener la informacion de la noticia 530
No se ha podido obtener la informacion de la noticia 530
No se ha podido obtener la informacion de la noticia 559
No se ha podido obtener la informacion de la noticia 566


In [10]:
#bulos

In [8]:
len(bulos)

5640

In [9]:
with open('fakenewsMaldita.json', 'w') as fp:
    json.dump(bulos, fp)